In [1]:
computer = 'win'

if computer == 'mac':
    str_chromedriver = '../../../chromedriver'
elif computer == 'win':
    str_chromedriver = '../../../chromedriver.exe'
else:
    print('？？？')
    exit(0)

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy
import UA

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./kit.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Kit_Info'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_Selenium())
option.add_argument('user-agent=' + UA.get_User_Agent())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                browser = Chrome(service=Service(str_chromedriver), options=option)

                browser.maximize_window()
                
                # = = = = = = = = = = = = = = =
                
                browser.get(url)

                time.sleep(3)
                
                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@aria-label, "More Information for")]')))
                list_oe = browser.find_elements(by=By.XPATH, value='//section[contains(@aria-label, "Alternate/OEM Part Number")]')

                oe = ''
                if len(list_oe) != 0:
                    for _ in list_oe:
                        if _.text.startswith('{Alternate Inventory Numbers: '):
                            oe += _.text.replace('{Alternate Inventory Numbers: ', '').replace('}', '') + ';'
                        elif _.text.startswith('OEM / Interchange Numbers: '):
                            oe += _.text.replace('OEM / Interchange Numbers: ', '') + ';'

                oe = ';'.join(oe[:-1].split(', '))

                # = = = = = = = = = = = = = = =
                
                src = browser.find_elements(by=By.XPATH, value='//img[@id="inlineimg[1]"]')
                if len(src) == 0:
                    src = ''
                else:
                    src = src[0].get_attribute('src')
                    
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Kit_Info': url,
                                         'Kit_OE': oe,
                                         'Kit_Src': src}])
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                browser.quit()
                time.sleep(3)
                continue
                
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Kit_Info': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./kit_oe_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：2439

https://www.rockauto.com/en/moreinfo.php?pk=8306460&cc=0&pt=5724  <->  [ok] - 剩余数量：2424
https://www.rockauto.com/en/moreinfo.php?pk=8307496&cc=0&pt=5728  <->  [ok] - 剩余数量：2423
https://www.rockauto.com/en/moreinfo.php?pk=8921164&cc=0&pt=5724  <->  [ok] - 剩余数量：2422
https://www.rockauto.com/en/moreinfo.php?pk=9659108&cc=0&pt=5736  <->  [ok] - 剩余数量：2421
https://www.rockauto.com/en/moreinfo.php?pk=8307492&cc=0&pt=5728  <->  [ok] - 剩余数量：2420
https://www.rockauto.com/en/moreinfo.php?pk=8307124&cc=0&pt=5728  <->  [ok] - 剩余数量：2419
https://www.rockauto.com/en/moreinfo.php?pk=9659104&cc=0&pt=5736  <->  [ok] - 剩余数量：2418
https://www.rockauto.com/en/moreinfo.php?pk=9722108&cc=0&pt=5728  <->  [ok] - 剩余数量：2417
https://www.rockauto.com/en/moreinfo.php?pk=8308216&cc=0&pt=5736  <->  [ok] - 剩余数量：2416
https://www.rockauto.com/en/moreinfo.php?pk=8921288&cc=0&pt=5736  <->  [ok] - 剩余数量：2415
https://www.rockauto.com/en/moreinfo.php?pk=8308212&cc=0&pt=5736  <->  [ok] - 剩余数量：2414
https://www.rockauto.c

https://www.rockauto.com/en/moreinfo.php?pk=9659088&cc=0&pt=5736  <->  [ok] - 剩余数量：2331
https://www.rockauto.com/en/moreinfo.php?pk=9659040&cc=0&pt=5724  <->  [ok] - 剩余数量：2330
https://www.rockauto.com/en/moreinfo.php?pk=9659092&cc=0&pt=5736  <->  [ok] - 剩余数量：2329
https://www.rockauto.com/en/moreinfo.php?pk=9659100&cc=0&pt=5728  <->  [ok] - 剩余数量：2328
https://www.rockauto.com/en/moreinfo.php?pk=9659204&cc=0&pt=5723  <->  [ok] - 剩余数量：2327
https://www.rockauto.com/en/moreinfo.php?pk=9659220&cc=0&pt=5722  <->  [ok] - 剩余数量：2326
https://www.rockauto.com/en/moreinfo.php?pk=9659096&cc=0&pt=5728  <->  [ok] - 剩余数量：2325
https://www.rockauto.com/en/moreinfo.php?pk=10454548&cc=0&pt=5728  <->  [ok] - 剩余数量：2324
https://www.rockauto.com/en/moreinfo.php?pk=10454556&cc=0&pt=5728  <->  [ok] - 剩余数量：2323
https://www.rockauto.com/en/moreinfo.php?pk=8921172&cc=0&pt=5724  <->  [ok] - 剩余数量：2322
https://www.rockauto.com/en/moreinfo.php?pk=8921176&cc=0&pt=5724  <->  [ok] - 剩余数量：2321
https://www.rockauto.com/en/mo

https://www.rockauto.com/en/moreinfo.php?pk=14645724&cc=0&pt=2208  <->  [ok] - 剩余数量：2238
https://www.rockauto.com/en/moreinfo.php?pk=8921180&cc=0&pt=5724  <->  [ok] - 剩余数量：2237
https://www.rockauto.com/en/moreinfo.php?pk=8304556&cc=0&pt=5724  <->  [ok] - 剩余数量：2236
https://www.rockauto.com/en/moreinfo.php?pk=8304560&cc=0&pt=5724  <->  [ok] - 剩余数量：2235
https://www.rockauto.com/en/moreinfo.php?pk=8307532&cc=0&pt=5728  <->  [ok] - 剩余数量：2234
https://www.rockauto.com/en/moreinfo.php?pk=8307536&cc=0&pt=5728  <->  [ok] - 剩余数量：2233
https://www.rockauto.com/en/moreinfo.php?pk=8307528&cc=0&pt=5728  <->  [ok] - 剩余数量：2232
https://www.rockauto.com/en/moreinfo.php?pk=8308248&cc=0&pt=5736  <->  [ok] - 剩余数量：2231
https://www.rockauto.com/en/moreinfo.php?pk=8308252&cc=0&pt=5736  <->  [ok] - 剩余数量：2230
https://www.rockauto.com/en/moreinfo.php?pk=8308256&cc=0&pt=5736  <->  [ok] - 剩余数量：2229
https://www.rockauto.com/en/moreinfo.php?pk=8528000&cc=0&pt=11860  <->  [ok] - 剩余数量：2228
https://www.rockauto.com/en/mo

https://www.rockauto.com/en/moreinfo.php?pk=8304568&cc=0&pt=5724  <->  [ok] - 剩余数量：2144
https://www.rockauto.com/en/moreinfo.php?pk=8307540&cc=0&pt=5728  <->  [ok] - 剩余数量：2143
https://www.rockauto.com/en/moreinfo.php?pk=8307544&cc=0&pt=5728  <->  [ok] - 剩余数量：2142
https://www.rockauto.com/en/moreinfo.php?pk=8307548&cc=0&pt=5728  <->  [ok] - 剩余数量：2141
https://www.rockauto.com/en/moreinfo.php?pk=8308264&cc=0&pt=5736  <->  [ok] - 剩余数量：2140
https://www.rockauto.com/en/moreinfo.php?pk=8308268&cc=0&pt=5736  <->  [ok] - 剩余数量：2139
https://www.rockauto.com/en/moreinfo.php?pk=8307556&cc=0&pt=5728  <->  [ok] - 剩余数量：2138
https://www.rockauto.com/en/moreinfo.php?pk=8313592&cc=0&pt=5723  <->  [ok] - 剩余数量：2137
https://www.rockauto.com/en/moreinfo.php?pk=8921324&cc=0&pt=5728  <->  [ok] - 剩余数量：2136
https://www.rockauto.com/en/moreinfo.php?pk=8304564&cc=0&pt=5724  <->  [ok] - 剩余数量：2135
https://www.rockauto.com/en/moreinfo.php?pk=8304560&cc=0&pt=5724  <->  [ok] - 剩余数量：2134
https://www.rockauto.com/en/more

https://www.rockauto.com/en/moreinfo.php?pk=8304568&cc=0&pt=5724  <->  [ok] - 剩余数量：2050
https://www.rockauto.com/en/moreinfo.php?pk=8921264&cc=0&pt=5728  <->  [ok] - 剩余数量：2049
https://www.rockauto.com/en/moreinfo.php?pk=8307548&cc=0&pt=5728  <->  [ok] - 剩余数量：2048
https://www.rockauto.com/en/moreinfo.php?pk=8308264&cc=0&pt=5736  <->  [ok] - 剩余数量：2047
https://www.rockauto.com/en/moreinfo.php?pk=8307552&cc=0&pt=5728  <->  [ok] - 剩余数量：2046
https://www.rockauto.com/en/moreinfo.php?pk=8308268&cc=0&pt=5736  <->  [ok] - 剩余数量：2045
https://www.rockauto.com/en/moreinfo.php?pk=13060841&cc=0&pt=5723  <->  [ok] - 剩余数量：2044
https://www.rockauto.com/en/moreinfo.php?pk=8921324&cc=0&pt=5728  <->  [ok] - 剩余数量：2043
https://www.rockauto.com/en/moreinfo.php?pk=8304576&cc=0&pt=5188  <->  [ok] - 剩余数量：2042
https://www.rockauto.com/en/moreinfo.php?pk=8304572&cc=0&pt=5724  <->  [ok] - 剩余数量：2041
https://www.rockauto.com/en/moreinfo.php?pk=8308284&cc=0&pt=10709  <->  [ok] - 剩余数量：2040
https://www.rockauto.com/en/mo

...

https://www.rockauto.com/en/moreinfo.php?pk=10519440&cc=0&pt=5728  <->  [ok] - 剩余数量：363
https://www.rockauto.com/en/moreinfo.php?pk=10890344&cc=0&pt=5736  <->  [ok] - 剩余数量：362
https://www.rockauto.com/en/moreinfo.php?pk=10890336&cc=0&pt=5728  <->  [ok] - 剩余数量：361
https://www.rockauto.com/en/moreinfo.php?pk=8306456&cc=0&pt=5568  <->  [ok] - 剩余数量：360
https://www.rockauto.com/en/moreinfo.php?pk=8306468&cc=0&pt=5724  <->  [ok] - 剩余数量：359
https://www.rockauto.com/en/moreinfo.php?pk=8307460&cc=0&pt=5728  <->  [ok] - 剩余数量：358
https://www.rockauto.com/en/moreinfo.php?pk=8308164&cc=0&pt=5728  <->  [ok] - 剩余数量：357
https://www.rockauto.com/en/moreinfo.php?pk=8306464&cc=0&pt=5724  <->  [ok] - 剩余数量：356
https://www.rockauto.com/en/moreinfo.php?pk=8307952&cc=0&pt=11860  <->  [ok] - 剩余数量：355
https://www.rockauto.com/en/moreinfo.php?pk=12804173&cc=0&pt=5736  <->  [ok] - 剩余数量：354
https://www.rockauto.com/en/moreinfo.php?pk=8308216&cc=0&pt=5736  <->  [ok] - 剩余数量：353
https://www.rockauto.com/en/moreinfo.p

https://www.rockauto.com/en/moreinfo.php?pk=10890316&cc=0&pt=5724  <->  [ok] - 剩余数量：269
https://www.rockauto.com/en/moreinfo.php?pk=9659128&cc=0&pt=11860  <->  [ok] - 剩余数量：268
https://www.rockauto.com/en/moreinfo.php?pk=9659132&cc=0&pt=5728  <->  [ok] - 剩余数量：267
https://www.rockauto.com/en/moreinfo.php?pk=9659136&cc=0&pt=5736  <->  [ok] - 剩余数量：266
https://www.rockauto.com/en/moreinfo.php?pk=8307488&cc=0&pt=5728  <->  [ok] - 剩余数量：265
https://www.rockauto.com/en/moreinfo.php?pk=8314712&cc=0&pt=5722  <->  [ok] - 剩余数量：264
https://www.rockauto.com/en/moreinfo.php?pk=9659044&cc=0&pt=5568  <->  [ok] - 剩余数量：263
https://www.rockauto.com/en/moreinfo.php?pk=9659048&cc=0&pt=5724  <->  [ok] - 剩余数量：262
https://www.rockauto.com/en/moreinfo.php?pk=9659068&cc=0&pt=5728  <->  [ok] - 剩余数量：261
https://www.rockauto.com/en/moreinfo.php?pk=8308244&cc=0&pt=5736  <->  [ok] - 剩余数量：260
https://www.rockauto.com/en/moreinfo.php?pk=9659072&cc=0&pt=11859  <->  [ok] - 剩余数量：259
https://www.rockauto.com/en/moreinfo.php

https://www.rockauto.com/en/moreinfo.php?pk=8306624&cc=0&pt=5724  <->  [ok] - 剩余数量：174
https://www.rockauto.com/en/moreinfo.php?pk=8308168&cc=0&pt=5728  <->  [ok] - 剩余数量：173
https://www.rockauto.com/en/moreinfo.php?pk=8308172&cc=0&pt=5736  <->  [ok] - 剩余数量：172
https://www.rockauto.com/en/moreinfo.php?pk=8307464&cc=0&pt=5728  <->  [ok] - 剩余数量：171
https://www.rockauto.com/en/moreinfo.php?pk=8308180&cc=0&pt=11860  <->  [ok] - 剩余数量：170
https://www.rockauto.com/en/moreinfo.php?pk=3950223&cc=0&pt=5722  <->  [ok] - 剩余数量：169
https://www.rockauto.com/en/moreinfo.php?pk=8306032&cc=0&pt=5724  <->  [ok] - 剩余数量：168
https://www.rockauto.com/en/moreinfo.php?pk=7093424&cc=0&pt=5728  <->  [ok] - 剩余数量：167
https://www.rockauto.com/en/moreinfo.php?pk=8307400&cc=0&pt=5728  <->  [ok] - 剩余数量：166
https://www.rockauto.com/en/moreinfo.php?pk=8921272&cc=0&pt=11859  <->  [ok] - 剩余数量：165
https://www.rockauto.com/en/moreinfo.php?pk=8308168&cc=0&pt=5728  <->  [ok] - 剩余数量：164
https://www.rockauto.com/en/moreinfo.php?

https://www.rockauto.com/en/moreinfo.php?pk=10890336&cc=0&pt=5728  <->  [ok] - 剩余数量：79
https://www.rockauto.com/en/moreinfo.php?pk=8306216&cc=0&pt=5724  <->  [ok] - 剩余数量：78
https://www.rockauto.com/en/moreinfo.php?pk=8310284&cc=0&pt=5564  <->  [ok] - 剩余数量：77
https://www.rockauto.com/en/moreinfo.php?pk=8314668&cc=0&pt=5723  <->  [ok] - 剩余数量：76
https://www.rockauto.com/en/moreinfo.php?pk=14048021&cc=0&pt=12910  <->  [ok] - 剩余数量：75
https://www.rockauto.com/en/moreinfo.php?pk=8306216&cc=0&pt=5724  <->  [ok] - 剩余数量：74
https://www.rockauto.com/en/moreinfo.php?pk=8308104&cc=0&pt=5736  <->  [ok] - 剩余数量：73
https://www.rockauto.com/en/moreinfo.php?pk=8310164&cc=0&pt=5564  <->  [ok] - 剩余数量：72
https://www.rockauto.com/en/moreinfo.php?pk=8314668&cc=0&pt=5723  <->  [ok] - 剩余数量：71
https://www.rockauto.com/en/moreinfo.php?pk=8314832&cc=0&pt=5722  <->  [ok] - 剩余数量：70
https://www.rockauto.com/en/moreinfo.php?pk=12804341&cc=0&pt=18586  <->  [ok] - 剩余数量：69
https://www.rockauto.com/en/moreinfo.php?pk=83146

D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
